In [1]:
import numpy as np
import csv

class reader(object):
  def __init__(self, data_file = "./data/wdbc.data"):
    self.value = []
    with open(data_file, "rb") as f:
      csv_reader = csv.reader(f, delimiter=",")
      for i, row in enumerate(csv_reader):
        self.value.append(row)
    # print self.value[0]
    # print self.value[0][0]
    # print self.value[0][1]

    self.raw_to_vector(self.value)
    self.split(num_validation_examples=114)
    
#     print self.id.shape
#     print len(self.x)
#     print self.y.shape
    
    self.num_examples = len(self.x_train)
    self.start_index = 0
    self.shuffle_indices = range(self.num_examples)

    self.num_examples_val = len(self.x_val) # = 114 (validation_data)
    self.start_index_val = 0
    self.shuffle_indices_val = range(self.num_examples_val)
    print self.num_examples_val

  def raw_to_vector(self, value):
    self.id = []
    self.x = []
    self.y = []

    for row in self.value:
        x = np.zeros(30)
        for i in range(30):
            x[i] = float(row[i+2])
        if row[1] == "B":
            y = 0
        else:
            y = 1
        self.x.append(x)
        self.y.append(y)
        id = int(row[0])
        self.id.append(id)
        
    self.x, self.y, self.id = np.array(self.x), np.array(self.y), np.array(self.id)

  def split(self, num_validation_examples):
    self.x_train = self.x[ num_validation_examples: ]
    self.x_val = self.x[ : num_validation_examples ]

    self.y_train = self.y[ num_validation_examples: ]
    self.y_val = self.y[ : num_validation_examples ]

    self.id_train = self.id[ num_validation_examples: ]
    self.id_val = self.id[ :num_validation_examples ]

  def next_batch(self, batch_size, split="train"):

    if split == "train":
      if self.start_index == 0:
        np.random.shuffle(self.shuffle_indices) # shuffle indices

      end_index = min([self.num_examples, self.start_index + batch_size])
      batch_indices = [ self.shuffle_indices[idx] for idx in range(self.start_index, end_index) ]

      batch_x = self.x_train[ batch_indices ]
      batch_y = self.y_train[ batch_indices ]
      batch_id = self.id_train[ batch_indices ] 

      if end_index == self.num_examples:
        self.start_index = 0
      else: self.start_index = end_index

      return batch_x, batch_y, batch_id 

    elif split == "val":
      if self.start_index_val == 0:
        np.random.shuffle(self.shuffle_indices_val) # shuffle indices

      end_index = min([self.num_examples_val, self.start_index_val + batch_size])
      batch_indices = [ self.shuffle_indices_val[idx] for idx in range(self.start_index_val, end_index) ]

      batch_x = self.x_val[ batch_indices ]
      batch_y = self.y_val[ batch_indices ]
      batch_id = self.id_val[ batch_indices ] 

      if end_index == self.num_examples_val:
        self.start_index_val = 0
      else: self.start_index_val = end_index

      return batch_x, batch_y, batch_id  


In [2]:
import tensorflow as tf

class dnn(object):
  def __init__(self):
    # Define model's input as tf.placeholder
    self.x = tf.placeholder(dtype=tf.float32, shape=[None, 30])
    self.y = tf.placeholder(dtype=tf.int32, shape=[None])

    
    # Define parameters for 3-layers dnn model
    self.w_1, self.b_1 = tf.get_variable(name="w_1", shape=[30, 50]), tf.get_variable(name="b_1", shape=[50])
    self.w_2, self.b_2 = tf.get_variable(name="w_2", shape=[50, 50]), tf.get_variable(name="b_2", shape=[50])
    self.w_3, self.b_3 = tf.get_variable(name="w_3", shape=[50, 2]), tf.get_variable(name="b_3", shape=[2])
    self.build_graph()

  def build_graph(self):

    # h1.shape = (batch, 50)
    h1 = tf.matmul(self.x, self.w_1) + self.b_1 
    h1 = tf.nn.relu(h1)

    # h2.shape = (batch, 50)
    h2 = tf.matmul(h1, self.w_2) + self.b_2
    h2 = tf.nn.relu(h2)

    # h3.shape = (batch, 2)
    h3 = tf.matmul(h2, self.w_3) + self.b_3
    h3 = tf.nn.softmax(h3, dim=-1)

    prediction = h3

    y_onehot = tf.one_hot(
      indices=self.y,
      depth=2,
      on_value=1.0,
      off_value=0.0,
      )

    # We use L2 loss as cost function and average the batch's loss
    self.loss = tf.reduce_mean((y_onehot - prediction) ** 2)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    
    self.train_op = optimizer.minimize(self.loss)

    pred_index = tf.cast(tf.argmax(prediction, dimension=1), tf.int32)
    correct_prediction = tf.equal(pred_index, self.y)
    self.acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print self.acc

In [3]:
import tensorflow as tf

# Loading data reader at Practice(2)
data_reader = reader()

# Loading DNN graph at Practice(3)
model = dnn()

# Define Session for running graph
# and initialize model's parameters
sess = tf.Session()

sess.run(tf.global_variables_initializer())

batch_size = 16
max_steps = 10000


for i in range(max_steps):
  # For each iteration, first we get batch x&y data
  x_train, y_train, id_train = data_reader.next_batch(16)

  # Next, construct feed for model's placeholder
  # feed is dictionary whose key is placeholder, and value is feeded value(numpy array)
  feed = {model.x: x_train, model.y: y_train}

  # Go training via running train_op with feeded data!
  # We run simultaneously train_op(backprop) and loss value
  _, loss = sess.run([model.train_op, model.loss], feed_dict=feed)


  # print loss stat every 100 iterations
  if i%100 == 0:
    print "| steps %07d | loss: %.3lf" % (i, loss)

#   running validation process every 100 iterations
  if i%100 == 0:
    x_val, y_val, id_val = data_reader.next_batch(50, split="val")
    feed_val = {model.x: x_val, model.y: y_val}

    validation_acc= sess.run([model.acc], feed_dict=feed_val)

    # Writing validation phase summary
    print "| steps %07d | Validation Accuracy: %.3lf" % (i, validation_acc)
    print model.acc


114
Tensor("Mean_1:0", shape=(), dtype=float32)
| steps 0000000 | loss: 0.250


TypeError: float argument required, not list